# This notebook demonstrate how to visualize values on USA map

The data was taken and from the drivendata competition:   
https://www.drivendata.org/competitions/143/tick-tick-bloom/

**This challenge was created on behalf of NASA**


In [12]:
# !pip install planetary_computer
# !pip install rioxarray
# !pip install odc-stac
# !pip install geopandas
# !pip install contextily

In [13]:
# %load_ext lab_black
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
import planetary_computer as pc
import cv2
from datetime import timedelta
import rioxarray
from IPython.display import Image
from PIL import Image as PILImage
from pathlib import Path
from tqdm import tqdm
import geopy.distance as distance
import pystac_client
import planetary_computer
import odc.stac
# Establish a connection to the STAC API
from pystac_client import Client
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import contextily as cx
import geopandas as gpd
from shapely.geometry import Point
import folium
import seaborn as sns

In [15]:

catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1", modifier=pc.sign_inplace
)


from pystac.extensions.eo import EOExtension as eo
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [16]:
DATA_DIR = Path.cwd().parent.resolve() / "data/final/public"

from google.colab import drive
drive.mount('/content/drive')

# path = "C:/Users/dalit/OneDrive/Documents/Projects/DS project/datadriven/algal_bloom/"
path="/content/drive/MyDrive/datadriven/algal_bloom/"
path

Mounted at /content/drive


'/content/drive/MyDrive/datadriven/algal_bloom/'

In [17]:
t="merge_save.csv"
merge_save= pd.read_csv(path + t,sep=",")
merge_save["severity"].unique()
merge_save
merge_save.date = pd.to_datetime(merge_save.date)
merge_save["year"] = merge_save.date.dt.year
merge_save

<ipython-input-17-5df2de94c882>:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  merge_save.date = pd.to_datetime(merge_save.date)


,uid,latitude,longitude,date,split,season,region,severity,density,year
0,aabm,39.080319,-86.430867,2018-05-14,train,5,1,1,585.0,2018
1,aacd,35.875083,-78.878434,2020-11-19,train,11,4,1,290.0,2020
2,aaee,35.487000,-79.062133,2016-08-24,train,8,4,1,1614.0,2016
3,aaff,38.049471,-99.827001,2019-07-23,train,7,1,3,111825.0,2019
4,aafl,39.474744,-86.898353,2021-08-23,train,8,1,4,2017313.0,2021
...,...,...,...,...,...,...,...,...,...,...
17055,zzsv,38.707825,-75.080867,2018-06-27,train,6,4,3,113125.0,2018
17056,zzuq,35.794000,-79.015368,2015-06-08,train,8,4,3,175726.0,2015
17057,zzwo,39.792190,-99.971050,2017-06-19,train,6,1,2,48510.0,2017
17058,zzwq,35.794000,-79.012551,2015-03-24,train,3,4,1,1271.0,2015


In [18]:
merge_save.groupby("year").describe()

latitude                                                        \
        count       mean       std        min        25%        50%   
year                                                                  
2013   1362.0  37.648136  2.968106  32.384010  35.794000  36.045000   
2014   1504.0  37.284307  2.069168  35.228000  35.794000  36.055000   
2015   2299.0  36.909668  1.998438  32.437490  35.671230  35.865953   
2016   2428.0  36.784627  1.898801  35.336013  35.678210  35.742000   
2017   2435.0  38.265552  3.329101  26.389430  35.790000  37.497200   
2018   1940.0  37.296716  2.220014  29.268433  35.722071  36.040000   
2019   1788.0  37.528497  2.301230  35.228000  35.718398  36.030000   
2020   1706.0  36.985468  1.757533  35.640000  35.790000  36.030000   
2021   1598.0  37.053644  1.852048  33.319170  35.686580  35.980000   

                           longitude             ... severity      density  \
            75%        max     count       mean  ...      75%  max   count   
year                                             ...                         
2013  38.057700  45.750000    1362.0 -89.303247  ...      3.0  5.0  1362.0   
2014  37.979200  45.750000    1504.0 -94.543015  ...      4.0  5.0  1504.0   
2015  37.667800  45.750000    2299.0 -88.756749  ...      3.0  5.0  2299.0   
2016  37.610161  45.750000    2428.0 -87.606782  ...      3.0  5.0  2428.0   
2017  39.808825  48.907060    2435.0 -91.683405  ...      3.0  5.0  2435.0   
2018  38.466255  45.750000    1940.0 -89.159862  ...      3.0  5.0  1940.0   
2019  38.718187  45.750000    1788.0 -89.686530  ...      3.0  5.0  1788.0   
2020  37.666700  44.568486    1706.0 -88.159758  ...      3.0  5.0  1706.0   
2021  37.979200  44.379810    1598.0 -89.788233  ...      3.0  5.0  1598.0   

                                                                       \
              mean           std        min           25%         50%   
year                                                                    
2013  9.450832e+05  5.592679e+06   0.000000   1791.398148  12604.5000   
2014  1.690942e+06  3.552182e+06  15.033333   4357.000000  41434.7698   
2015  1.285603e+06  1.699406e+07   0.000000   3059.000000  25044.0000   
2016  6.447589e+05  1.661463e+06   0.000000   3593.000000  26048.5000   
2017  9.048520e+05  1.991004e+06   0.000000  10089.000000  61247.0000   
2018  1.143891e+06  3.067177e+06   0.000000   2905.000000  21488.5000   
2019  1.212320e+06  2.726894e+06   0.000000   4418.500000  39145.5000   
2020  9.474777e+05  2.097337e+06   0.000000   5335.000000  31950.0000   
2021  1.109918e+06  3.114032e+06   0.000000   6679.500000  51582.5000   

                                 
              75%           max  
year                             
2013   141978.000  1.959300e+08  
2014  2657883.125  9.127360e+07  
2015   465157.000  8.046675e+08  
2016   225897.750  3.196620e+07  
2017   566230.000  3.581020e+07  
2018   339209.250  5.969250e+07  
2019   879234.125  4.260262e+07  
2020   384013.250  2.183120e+07  
2021   509929.250  7.181390e+07  

[9 rows x 48 columns]

In [19]:
dict={1:1,2:2,11:1,12:1,3:2,4:2,5:2,6:3,7:3,8:3,9:4,10:4}
data=merge_save.copy()
# print(data)
data['seas']=merge_save['season'].map(dict)

coord=[merge_save["latitude"][0],merge_save["longitude"][0]]

winter=data[data['seas']==1]
spring=data[data['seas']==2]
summer=data[data['seas']==3]
autumn=data[data['seas']==4]
spring.groupby("year").describe()
spring_2013=spring[spring['year']==2013]
spring_2015=spring[spring['year']==2015]
spring_2020=spring[spring['year']==2020]
spring_2016=spring[spring['year']==2016]

summer_2013=summer[summer['year']==2013]
summer_2015=summer[summer['year']==2015]
summer_2016=summer[summer['year']==2016]
summer_2020=summer[summer['year']==2020]

autumn_2013=autumn[autumn['year']==2013]
autumn_2015=autumn[autumn['year']==2015]
autumn_2016=autumn[autumn['year']==2016]
autumn_2020=autumn[autumn['year']==2020]

winter_2013=winter[winter['year']==2013]
winter_2015=winter[winter['year']==2015]
winter_2016=winter[winter['year']==2016]
winter_2020=winter[winter['year']==2020]




# First option for presenting the values on the maps

In [20]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

us_cities = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv")

fig = px.scatter_mapbox(summer_2013, lat="latitude", lon="longitude",color=summer_2013["severity"],
                        # cmap=plt.cm.get_cmap('plasma', 5)
                        zoom=2,
                        size_max=10,
                        width = 600,
                        height=500)
fig.update_layout(
    mapbox_style=
    "open-street-map",

      )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
fig = px.scatter_mapbox(summer_2015, lat="latitude", lon="longitude",color=summer_2015["severity"],
                        # cmap=plt.cm.get_cmap('plasma', 5)
                        zoom=2,
                        size_max=10,
                        width = 600,
                        height=500)
fig.update_layout(
    mapbox_style=
    "open-street-map",

      )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()




fig = px.scatter_mapbox(summer_2016, lat="latitude", lon="longitude",color=summer_2016["severity"],
                        # cmap=plt.cm.get_cmap('plasma', 5)
                        zoom=2,
                        size_max=10,
                        width = 600,
                        height=500)
fig.update_layout(
    mapbox_style=
    "open-street-map",

      )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

fig = px.scatter_mapbox(summer_2020, lat="latitude", lon="longitude",color=summer_2020["severity"],
                        # cmap=plt.cm.get_cmap('plasma', 5)
                        zoom=2,
                        size_max=10,
                        width = 600,
                        height=500)
fig.update_layout(
    mapbox_style=
    "open-street-map",

      )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# Second option for presenting values on maps

In [24]:
import pandas as pd
import plotly.express as px

us_cities = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv")


fig = px.scatter_mapbox(spring, lat="latitude", lon="longitude",color=spring["severity"],
                        # cmap=plt.cm.get_cmap('plasma', 5)
                        zoom=3,
                        size_max=10,
                        height=300)
fig.update_layout(
    mapbox_style=
    # "stamen- terrain",
    "white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()




fig = px.scatter_mapbox(summer, lat="latitude", lon="longitude",color=summer["severity"],
                        # cmap=plt.cm.get_cmap('plasma', 5)
                        zoom=3,
                        size_max=10,
                        height=400)
fig.update_layout(
    mapbox_style=
    # "stamen- terrain",
    "white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()